In [1]:
import numpy as np
import pandas as pd
import os
import io
import seaborn as sns
import requests
import json

import matplotlib.pyplot as plt
import pymc3 as pm

import scipy
import scipy.stats as stats
import scipy.optimize as opt
#import statsmodels.api as sm

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
plt.style.use('seaborn-darkgrid')
pd.set_option('display.max_columns', 500)
%matplotlib inline 

In [307]:
#Get NHL Data for Last Season
r = requests.get('https://statsapi.web.nhl.com/api/v1/schedule?stats=gameLog&season=20182019')
data = r.json()

In [308]:
gameType = []
away = []
home = []
away_score = []
home_score = []
venue = []
gameDate = []
gameState = []
gameid = []

In [309]:
for date in data['dates']:
    for game in date['games']:
        gameType.append(game['gameType'])
        away.append(game['teams']['away']['team']['name'])
        home.append(game['teams']['home']['team']['name'])
        away_score.append(game['teams']['away']['score'])
        home_score.append(game['teams']['home']['score'])
        venue.append(game['venue']['name'])
        gameDate.append(str(date['date']))
        gameState.append(game['status']['statusCode'])
        gameid.append(game['gamePk'])
    print("Done with " + str(date['date']))

Done with 2018-09-15
Done with 2018-09-16
Done with 2018-09-17
Done with 2018-09-18
Done with 2018-09-19
Done with 2018-09-20
Done with 2018-09-21
Done with 2018-09-22
Done with 2018-09-23
Done with 2018-09-24
Done with 2018-09-25
Done with 2018-09-26
Done with 2018-09-27
Done with 2018-09-28
Done with 2018-09-29
Done with 2018-09-30
Done with 2018-10-01
Done with 2018-10-03
Done with 2018-10-04
Done with 2018-10-05
Done with 2018-10-06
Done with 2018-10-07
Done with 2018-10-08
Done with 2018-10-09
Done with 2018-10-10
Done with 2018-10-11
Done with 2018-10-13
Done with 2018-10-14
Done with 2018-10-15
Done with 2018-10-16
Done with 2018-10-17
Done with 2018-10-18
Done with 2018-10-19
Done with 2018-10-20
Done with 2018-10-21
Done with 2018-10-22
Done with 2018-10-23
Done with 2018-10-24
Done with 2018-10-25
Done with 2018-10-26
Done with 2018-10-27
Done with 2018-10-28
Done with 2018-10-29
Done with 2018-10-30
Done with 2018-10-31
Done with 2018-11-01
Done with 2018-11-02
Done with 201

In [310]:
df = pd.DataFrame({'GameType': gameType, 'Away': away, 'Home': home, 'Final_Away_Score': away_score, 'Final_Home_Score': home_score,
                   'Venue': venue, 'GameDate': gameDate, 'GameState': gameState, 'GameID': gameid})

In [311]:
df['GameState'].unique()

array(['6', '7', '4', '3', '1'], dtype=object)

In [312]:
#Only for Current season
df = df[(df['GameState'] == '6') | (df['GameState'] == '7')]

In [313]:
df.tail()

,GameType,Away,Home,Final_Away_Score,Final_Home_Score,Venue,GameDate,GameState,GameID
544,R,Columbus Blue Jackets,Philadelphia Flyers,4,3,Wells Fargo Center,2018-12-06,7,2018020431
545,R,New York Islanders,Pittsburgh Penguins,2,6,PPG Paints Arena,2018-12-06,7,2018020432
546,R,Montréal Canadiens,Ottawa Senators,5,2,Canadian Tire Centre,2018-12-06,7,2018020433
547,R,Boston Bruins,Tampa Bay Lightning,2,3,Amalie Arena,2018-12-06,7,2018020434
548,R,Washington Capitals,Arizona Coyotes,4,2,Gila River Arena,2018-12-06,6,2018020436


In [314]:
'''
Alright we have all of the game results. Time to pull in all of the plays for EVERY game! :0
'''

'\nAlright we have all of the game results. Time to pull in all of the plays for EVERY game! :0\n'

In [315]:
columns = df.columns.tolist() + ['EventID', 'Time_in_Period', 'Time_Remaining', 'Period',
       'Period_Type', 'x', 'y', 'Event_Description', 'Event',
       'Event_Type_ID', 'Home_Score', 'Away_Score', 'Player_1_Name',
       'Player_1_ID', 'Player_2_Name', 'Player_2_ID', 'Event_Team', 'Goalie', 'Goalie_ID',
        'Shooter', 'Shooter_ID', 'Scorer', 'Scorer_ID', 'Assist1', 'Assist1_ID', 'Assist2', 'Assist2_ID',
        'EmptyNet']

In [316]:
master = pd.DataFrame(columns=columns)

In [317]:
n = 0
for game in df['GameID']:
    #Get Events for a game
    r = requests.get('https://statsapi.web.nhl.com/api/v1/game/'+ str(game) + '/feed/live')
    data = r.json()
    eventId = []
    periodTime = []
    periodTimeRemaining = []
    period = []
    periodType = []
    coord_x = []
    coord_y = []
    description = []
    event = []
    eventTypeID = []
    goals_home = []
    goals_away = []
    player_1 = []
    player_2 = []
    player_1_id = []
    player_2_id = []
    team = []
    goalie = []
    goalie_id = []
    shooter = []
    shooter_id = []
    scorer = []
    scorer_id = []
    assist_1 = []
    assist_1_id = []
    assist_2 = []
    assist_2_id = []
    empty_net = []
    shot_type = []
    
    for play in data['liveData']['plays']['allPlays']:
        try:
            eventId.append(play['about']['eventId'])
        except:
            eventId.append('None')
        try:
            periodTime.append(play['about']['periodTime'])
        except:
            periodTime.append(np.NaN)
        try:
            periodTimeRemaining.append(play['about']['periodTimeRemaining'])
        except:
            periodTimeRemaining.append(np.NaN)
        try:
            period.append(play['about']['period'])
        except:
            period.append('None')
        try:
            periodType.append(play['about']['periodType'])
        except:
            periodType.append('None')
        try:
            coord_x.append(play['coordinates']['x'])
        except:
            coord_x.append(np.NaN)
        try:
            coord_y.append(play['coordinates']['y'])
        except:
            coord_y.append(np.NaN)
        try:
            description.append(play['result']['description'])
        except:
            description.append('None')
        try:
            event.append(play['result']['event'])
        except:
            event.append('None')
        try:
            eventTypeID.append(play['result']['eventTypeId'])
        except:
            eventTypeID.append('None')
        try:
            goals_home.append(play['about']['goals']['home'])
        except:
            goals_home.append(np.NaN)
        try:
            goals_away.append(play['about']['goals']['away'])
        except:
            goals_away.append(np.NaN)
        try:
            player_1.append(play['players'][0]['player']['fullName'])
        except:
            player_1.append('None')
        try:
            player_1_id.append(play['players'][0]['player']['id'])
        except:
            player_1_id.append('None')
        try:
            player_2.append(play['players'][1]['player']['fullName'])
        except:
            player_2.append('None')
        try:
            player_2_id.append(play['players'][1]['player']['id'])
        except:
            player_2_id.append('None')
            
        #Append Empty Net Goal if Applicable (really just a safety collection)
        try:
            empty_net.append(play['result']['emptyNet'])
        except:
            empty_net.append('None')
            
        try:
            shot_type.append(play['result']['secondaryType'])
        except:
            shot_type.append('None')
            
        #Append Goalies if Applicable
        players = []
        players_id = []
        for i in range(0, 7):
            try:
                if play['players'][i]['playerType'] == 'Goalie':
                    players.append(play['players'][i]['player']['fullName'])
                    players_id.append(play['players'][i]['player']['id'])
            except:
                pass
            
        if len(players) == 1:
            goalie.append(players[0])
            goalie_id.append(players_id[0])
        elif len(players) == 0:
            goalie.append('None')
            goalie_id.append('None')
        else:
            goalie.append(str(players))
            goalie_id.append(str(players_id))
        
        #Append Scorers if Applicable
        players = []
        players_id = []
        for i in range(0, 7):
            try:
                if play['players'][i]['playerType'] == 'Scorer':
                    players.append(play['players'][i]['player']['fullName'])
                    players_id.append(play['players'][i]['player']['id'])
            except:
                pass
            
        if len(players) == 1:
            scorer.append(players[0])
            scorer_id.append(players_id[0])
        elif len(players) == 0:
            scorer.append('None')
            scorer_id.append('None')
        else:
            scorer.append(str(players))
            scorer_id.append(str(players_id))
        
        #Append Shooter if Applicable
        players = []
        players_id = []
        for i in range(0, 7):
            try:
                if play['players'][i]['playerType'] == 'Shooter':
                    players.append(play['players'][i]['player']['fullName'])
                    players_id.append(play['players'][i]['player']['id'])
            except:
                pass
            
        if len(players) == 1:
            shooter.append(players[0])
            shooter_id.append(players_id[0])
        elif len(players) == 0:
            shooter.append('None')
            shooter_id.append('None')
        else:
            shooter.append(str(players))
            shooter_id.append(str(players_id))
            
        #Append Assists if Applicable
        assists = []
        assists_id = []
        for i in range(0, 7):
            try:
                if play['players'][i]['playerType'] == 'Assist':
                    assists.append(play['players'][i]['player']['fullName'])
                    assists_id.append(play['players'][i]['player']['id'])
            except:
                pass
            
        if len(assists) > 0:
            assist_1.append(assists[0])
            assist_1_id.append(assists_id[0])

            if len(assists) > 1:
                assist_2.append(assists[1])
                assist_2_id.append(assists_id[1])
            else:
                assist_2.append('None')
                assist_2_id.append('None')
        else:
            assist_1.append('None')
            assist_1_id.append('None')
            assist_2.append('None')
            assist_2_id.append('None')

            
        try:
            team.append(play['team']['name'])
        except:
            team.append('None')
        
            
    plays_df = pd.DataFrame({'EventID': eventId, 'Time_in_Period': periodTime, 'Time_Remaining': periodTimeRemaining, 
                        'Period': period, 'Period_Type': periodType, 'x': coord_x, 'y': coord_y, 
                         'Event_Description': description, 'Event': event, 'Event_Type_ID': eventTypeID,
                        'Home_Score': goals_home, 'Away_Score': goals_away, 'Player_1_Name': player_1, 
                         'Player_1_ID': player_1_id, 'Player_2_Name': player_2, 'Player_2_ID': player_2_id, 'Event_Team': team, 
                            'Goalie': goalie, 'Goalie_ID': goalie_id, 'Shooter': shooter, 'Shooter_ID': shooter_id, 
                            'Scorer': scorer, 'Scorer_ID': scorer_id, 'Assist1': assist_1, 'Assist1_ID': assist_1_id,
                            'Assist2': assist_2, 'Assists2_ID': assist_2_id, 'ShotType': shot_type,'EmptyNet': empty_net})
    
    plays_df['GameID'] = game
    plays_df = plays_df.merge(df[df['GameID']==game], how='left', on='GameID')
    master = pd.concat([master, plays_df])
    print(str(game))
    print(n)
    n = n+1

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:221: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



2018010001
0
2018010002
1
2018010003
2
2018010004
3
2018010005
4
2018010006
5
2018010007
6
2018010008
7
2018010009
8
2018010010
9
2018010011
10
2018010012
11
2018010013
12
2018010014
13
2018010015
14
2018010016
15
2018010017
16
2018010018
17
2018010019
18
2018010020
19
2018010021
20
2018010022
21
2018010023
22
2018010024
23
2018010025
24
2018010026
25
2018010027
26
2018010028
27
2018010029
28
2018010030
29
2018010031
30
2018010032
31
2018010033
32
2018010034
33
2018010035
34
2018010036
35
2018010037
36
2018010038
37
2018010039
38
2018010040
39
2018010041
40
2018010042
41
2018010043
42
2018010044
43
2018010045
44
2018010046
45
2018010047
46
2018010048
47
2018010049
48
2018010050
49
2018010051
50
2018010052
51
2018010053
52
2018010054
53
2018010055
54
2018010056
55
2018010057
56
2018010058
57
2018010059
58
2018010060
59
2018010061
60
2018010062
61
2018010063
62
2018010064
63
2018010065
64
2018010066
65
2018010067
66
2018010068
67
2018010069
68
2018010070
69
2018010071
70
2018010072
71
20

In [318]:
master['ShotType'].value_counts()

None                                      133271
Wrist Shot                                 18087
Slap Shot                                   4939
Snap Shot                                   4750
Backhand                                    2666
Tip-In                                      1735
Tripping                                     711
Deflected                                    585
Hooking                                      561
Slashing                                     530
Interference                                 444
Roughing                                     369
Holding                                      367
Wrap-around                                  319
Hi-sticking                                  302
Fighting                                     213
Cross checking                               147
Delaying Game - Puck over glass              120
Too many men on the ice                      108
Boarding                                      68
Unsportsmanlike cond

In [319]:
r = requests.get('https://statsapi.web.nhl.com/api/v1/game/'+ str(2018020412) + '/feed/live')
data = r.json()

In [168]:
from pprint import pprint
for key, value in data.items():
    pprint(key)

'copyright'
'gamePk'
'link'
'metaData'
'gameData'
'liveData'


In [169]:
for key, value in data['liveData'].items():
    pprint(key)

'plays'
'linescore'
'boxscore'
'decisions'


In [320]:
rinkside_df = pd.DataFrame(columns = ['Period', 'RinkSide_Home', 'RinkSide_Away', 'GameID', 'Home', 'Away'])
for game in df['GameID']:
    #Get Events for a game
    r = requests.get('https://statsapi.web.nhl.com/api/v1/game/'+ str(game) + '/feed/live')
    data = r.json()
    
    period = []
    home = []
    away = []
    for p in data['liveData']['linescore']['periods']:
        try:
            period.append(p['num'])
        except:
            period.append('None')
        try:
            home.append(p['home']['rinkSide'])
        except:
            home.append('None')
        try:
            away.append(p['away']['rinkSide'])
        except:
            away.append('None')
            
    game_df = pd.DataFrame({'Period': period, 'RinkSide_Home': home, 'RinkSide_Away': away})
    game_df['GameID'] = game
    game_df['Home'] = data['liveData']['linescore']['teams']['home']['team']['name']
    game_df['Away'] = data['liveData']['linescore']['teams']['away']['team']['name']
    
    print("Done with " + str(game))
    rinkside_df = pd.concat([rinkside_df, game_df])

Done with 2018010001
Done with 2018010002
Done with 2018010003
Done with 2018010004
Done with 2018010005
Done with 2018010006
Done with 2018010007
Done with 2018010008
Done with 2018010009
Done with 2018010010
Done with 2018010011
Done with 2018010012
Done with 2018010013
Done with 2018010014
Done with 2018010015
Done with 2018010016
Done with 2018010017
Done with 2018010018
Done with 2018010019
Done with 2018010020
Done with 2018010021
Done with 2018010022
Done with 2018010023
Done with 2018010024
Done with 2018010025
Done with 2018010026
Done with 2018010027
Done with 2018010028
Done with 2018010029
Done with 2018010030
Done with 2018010031
Done with 2018010032
Done with 2018010033
Done with 2018010034
Done with 2018010035
Done with 2018010036
Done with 2018010037
Done with 2018010038
Done with 2018010039
Done with 2018010040
Done with 2018010041
Done with 2018010042
Done with 2018010043
Done with 2018010044
Done with 2018010045
Done with 2018010046
Done with 2018010047
Done with 201

Done with 2018020282
Done with 2018020283
Done with 2018020284
Done with 2018020285
Done with 2018020286
Done with 2018020287
Done with 2018020288
Done with 2018020289
Done with 2018020290
Done with 2018020291
Done with 2018020292
Done with 2018020293
Done with 2018020294
Done with 2018020295
Done with 2018020296
Done with 2018020297
Done with 2018020298
Done with 2018020299
Done with 2018020300
Done with 2018020301
Done with 2018020302
Done with 2018020303
Done with 2018020304
Done with 2018020305
Done with 2018020306
Done with 2018020307
Done with 2018020308
Done with 2018020309
Done with 2018020310
Done with 2018020311
Done with 2018020312
Done with 2018020313
Done with 2018020314
Done with 2018020315
Done with 2018020316
Done with 2018020317
Done with 2018020319
Done with 2018020320
Done with 2018020321
Done with 2018020322
Done with 2018020323
Done with 2018020318
Done with 2018020324
Done with 2018020325
Done with 2018020326
Done with 2018020327
Done with 2018020328
Done with 201

In [321]:
rinkside_df.head()

,Period,RinkSide_Home,RinkSide_Away,GameID,Home,Away
0,1,right,left,2018010001,Calgary Flames,Boston Bruins
1,2,left,right,2018010001,Calgary Flames,Boston Bruins
2,3,right,left,2018010001,Calgary Flames,Boston Bruins
3,4,left,right,2018010001,Calgary Flames,Boston Bruins
0,1,left,right,2018010002,Boston Bruins,Washington Capitals


In [322]:
home_rinkside = pd.melt(rinkside_df, id_vars=['GameID', 'Period', 'RinkSide_Home'], value_vars=['Home'])
home_rinkside.columns = ['GameID', 'Period', 'Side', 'Home_Away', 'Event_Team']
away_rinkside = pd.melt(rinkside_df, id_vars=['GameID', 'Period', 'RinkSide_Away'], value_vars=['Away'])
away_rinkside.columns = ['GameID', 'Period', 'Side', 'Home_Away', 'Event_Team']

In [323]:
rinkside_df2 = pd.concat([home_rinkside, away_rinkside])

In [324]:
master.head()

,Assist1,Assist1_ID,Assist2,Assist2_ID,Assists2_ID,Away,Away_Score,EmptyNet,Event,EventID,Event_Description,Event_Team,Event_Type_ID,Final_Away_Score,Final_Home_Score,GameDate,GameID,GameState,GameType,Goalie,Goalie_ID,Home,Home_Score,Period,Period_Type,Player_1_ID,Player_1_Name,Player_2_ID,Player_2_Name,Scorer,Scorer_ID,Shooter,Shooter_ID,ShotType,Time_Remaining,Time_in_Period,Venue,x,y
0,None,None,None,NaN,None,Washington Capitals,0,None,Game Scheduled,1,Game Scheduled,None,GAME_SCHEDULED,1,2,2018-09-16,2018010002,6,PR,None,None,Boston Bruins,0,1,REGULAR,None,None,None,None,None,None,None,None,None,20:00,00:00,TD Garden,NaN,NaN
1,None,None,None,NaN,None,Washington Capitals,0,None,Period Ready,5,Period Ready,None,PERIOD_READY,1,2,2018-09-16,2018010002,6,PR,None,None,Boston Bruins,0,1,REGULAR,None,None,None,None,None,None,None,None,None,20:00,00:00,TD Garden,NaN,NaN
2,None,None,None,NaN,None,Washington Capitals,0,None,Period Start,51,Period Start,None,PERIOD_START,1,2,2018-09-16,2018010002,6,PR,None,None,Boston Bruins,0,1,REGULAR,None,None,None,None,None,None,None,None,None,20:00,00:00,TD Garden,NaN,NaN
3,None,None,None,NaN,None,Washington Capitals,0,None,Faceoff,52,David Krejci faceoff won against Michael Sgarb...,Boston Bruins,FACEOFF,1,2,2018-09-16,2018010002,6,PR,None,None,Boston Bruins,0,1,REGULAR,8471276,David Krejci,8475958,Michael Sgarbossa,None,None,None,None,None,20:00,00:00,TD Garden,0.0,0.0
4,None,None,None,NaN,None,Washington Capitals,0,None,Giveaway,53,Giveaway by David Krejci,Boston Bruins,GIVEAWAY,1,2,2018-09-16,2018010002,6,PR,None,None,Boston Bruins,0,1,REGULAR,8471276,David Krejci,None,None,None,None,None,None,None,19:44,00:16,TD Garden,50.0,-29.0


In [278]:
for key in data['liveData']:
    print(key)

plays
linescore
boxscore
decisions


In [279]:
data['liveData']['plays']

{'allPlays': [{'result': {'event': 'Game Scheduled',
    'eventCode': 'NJD1',
    'eventTypeId': 'GAME_SCHEDULED',
    'description': 'Game Scheduled'},
   'about': {'eventIdx': 0,
    'eventId': 1,
    'period': 1,
    'periodType': 'REGULAR',
    'ordinalNum': '1st',
    'periodTime': '00:00',
    'periodTimeRemaining': '20:00',
    'dateTime': '2017-12-23T22:57:35Z',
    'goals': {'away': 0, 'home': 0}},
   'coordinates': {}},
  {'result': {'event': 'Period Ready',
    'eventCode': 'NJD5',
    'eventTypeId': 'PERIOD_READY',
    'description': 'Period Ready'},
   'about': {'eventIdx': 1,
    'eventId': 5,
    'period': 1,
    'periodType': 'REGULAR',
    'ordinalNum': '1st',
    'periodTime': '00:00',
    'periodTimeRemaining': '20:00',
    'dateTime': '2017-12-24T00:07:23Z',
    'goals': {'away': 0, 'home': 0}},
   'coordinates': {}},
  {'result': {'event': 'Period Start',
    'eventCode': 'NJD51',
    'eventTypeId': 'PERIOD_START',
    'description': 'Period Start'},
   'about': {'

In [76]:
data['liveData']['plays']['allPlays'][121]['players'][0]['playerType']

'Shooter'

In [80]:
master.head()

,Assist 1,Assist 2,Assist1_ID,Assists2_ID,Away,Away_Score,Event,EventID,Event_Description,Event_Team,Event_Type_ID,Final_Away_Score,Final_Home_Score,GameDate,GameID,GameState,GameType,Goalie,Goalie_ID,Home,Home_Score,Period,Period_Type,Player_1_ID,Player_1_Name,Player_2_ID,Player_2_Name,Scorer,Scorer_ID,Shooter,Shooter_ID,Time_Remaining,Time_in_Period,Venue,x,y
0,NaN,NaN,NaN,NaN,Washington Capitals,0,Game Scheduled,1,Game Scheduled,None,GAME_SCHEDULED,1,2,2018-09-16,2018010002,6,PR,NaN,NaN,Boston Bruins,0,1,REGULAR,None,None,None,None,NaN,NaN,NaN,NaN,20:00,00:00,TD Garden,NaN,NaN
1,NaN,NaN,NaN,NaN,Washington Capitals,0,Period Ready,5,Period Ready,None,PERIOD_READY,1,2,2018-09-16,2018010002,6,PR,NaN,NaN,Boston Bruins,0,1,REGULAR,None,None,None,None,NaN,NaN,NaN,NaN,20:00,00:00,TD Garden,NaN,NaN
2,NaN,NaN,NaN,NaN,Washington Capitals,0,Period Start,51,Period Start,None,PERIOD_START,1,2,2018-09-16,2018010002,6,PR,NaN,NaN,Boston Bruins,0,1,REGULAR,None,None,None,None,NaN,NaN,NaN,NaN,20:00,00:00,TD Garden,NaN,NaN
3,NaN,NaN,NaN,NaN,Washington Capitals,0,Faceoff,52,David Krejci faceoff won against Michael Sgarb...,Boston Bruins,FACEOFF,1,2,2018-09-16,2018010002,6,PR,NaN,NaN,Boston Bruins,0,1,REGULAR,8471276,David Krejci,8475958,Michael Sgarbossa,NaN,NaN,NaN,NaN,20:00,00:00,TD Garden,0.0,0.0
4,NaN,NaN,NaN,NaN,Washington Capitals,0,Giveaway,53,Giveaway by David Krejci,Boston Bruins,GIVEAWAY,1,2,2018-09-16,2018010002,6,PR,NaN,NaN,Boston Bruins,0,1,REGULAR,8471276,David Krejci,None,None,NaN,NaN,NaN,NaN,19:44,00:16,TD Garden,50.0,-29.0


In [325]:
def event2(row):
    if 'save' in row['Event_Description'].lower():
        return 'Save'
    else:
        return row['Event']

In [326]:
master['Event2'] = master.apply(lambda row: event2(row), axis=1)

In [327]:
#Add in Direction
master.head()

,Assist1,Assist1_ID,Assist2,Assist2_ID,Assists2_ID,Away,Away_Score,EmptyNet,Event,EventID,Event_Description,Event_Team,Event_Type_ID,Final_Away_Score,Final_Home_Score,GameDate,GameID,GameState,GameType,Goalie,Goalie_ID,Home,Home_Score,Period,Period_Type,Player_1_ID,Player_1_Name,Player_2_ID,Player_2_Name,Scorer,Scorer_ID,Shooter,Shooter_ID,ShotType,Time_Remaining,Time_in_Period,Venue,x,y,Event2
0,None,None,None,NaN,None,Washington Capitals,0,None,Game Scheduled,1,Game Scheduled,None,GAME_SCHEDULED,1,2,2018-09-16,2018010002,6,PR,None,None,Boston Bruins,0,1,REGULAR,None,None,None,None,None,None,None,None,None,20:00,00:00,TD Garden,NaN,NaN,Game Scheduled
1,None,None,None,NaN,None,Washington Capitals,0,None,Period Ready,5,Period Ready,None,PERIOD_READY,1,2,2018-09-16,2018010002,6,PR,None,None,Boston Bruins,0,1,REGULAR,None,None,None,None,None,None,None,None,None,20:00,00:00,TD Garden,NaN,NaN,Period Ready
2,None,None,None,NaN,None,Washington Capitals,0,None,Period Start,51,Period Start,None,PERIOD_START,1,2,2018-09-16,2018010002,6,PR,None,None,Boston Bruins,0,1,REGULAR,None,None,None,None,None,None,None,None,None,20:00,00:00,TD Garden,NaN,NaN,Period Start
3,None,None,None,NaN,None,Washington Capitals,0,None,Faceoff,52,David Krejci faceoff won against Michael Sgarb...,Boston Bruins,FACEOFF,1,2,2018-09-16,2018010002,6,PR,None,None,Boston Bruins,0,1,REGULAR,8471276,David Krejci,8475958,Michael Sgarbossa,None,None,None,None,None,20:00,00:00,TD Garden,0.0,0.0,Faceoff
4,None,None,None,NaN,None,Washington Capitals,0,None,Giveaway,53,Giveaway by David Krejci,Boston Bruins,GIVEAWAY,1,2,2018-09-16,2018010002,6,PR,None,None,Boston Bruins,0,1,REGULAR,8471276,David Krejci,None,None,None,None,None,None,None,19:44,00:16,TD Garden,50.0,-29.0,Giveaway


In [328]:
master = master.merge(rinkside_df2[['GameID', 'Event_Team', 'Period', 'Side']], how='left', on=['GameID', 'Event_Team', 'Period'])

In [329]:
master.to_csv("201819_nhl_actions_120618.csv", index=False)
#master.to_csv("201718_nhl_actions.csv", index=False)

In [330]:
# df_2019 = pd.read_csv("/users/alexlentz/Desktop/201819_nhl_actions_100218.csv")
# df_2018 = master.copy()

df_2018 = pd.read_csv("201718_nhl_actions.csv")
df_2019 = pd.read_csv("201819_nhl_actions_120618.csv")

In [331]:
df_2018['Season'] = '2017-2018'
df_2019['Season'] = '2018-2019'

In [346]:
#TEMPORARY
df_2018.drop("Side_x", axis=1, inplace=True)
df_2018.rename(columns={'Side_y': 'Side'}, inplace=True)

In [347]:
df_2018.columns

Index(['Assist1', 'Assist1_ID', 'Assist2', 'Assist2_ID', 'Assists2_ID', 'Away',
       'Away_Score', 'EmptyNet', 'Event', 'EventID', 'Event_Description',
       'Event_Team', 'Event_Type_ID', 'Final_Away_Score', 'Final_Home_Score',
       'GameDate', 'GameID', 'GameState', 'GameType', 'Goalie', 'Goalie_ID',
       'Home', 'Home_Score', 'Period', 'Period_Type', 'Player_1_ID',
       'Player_1_Name', 'Player_2_ID', 'Player_2_Name', 'Scorer', 'Scorer_ID',
       'Shooter', 'Shooter_ID', 'ShotType', 'Time_Remaining', 'Time_in_Period',
       'Venue', 'x', 'y', 'Event2', 'Side', 'Season'],
      dtype='object')

In [348]:
combined = pd.concat([df_2018, df_2019])

In [250]:
#combined.to_csv("combined_hockey2.csv")

In [349]:
shots = combined[combined['Event_Type_ID'] == 'SHOT']

In [350]:
avgs_df = shots[['Event_Team', 'GameID', 'Period', 'x', 'y']].groupby(['GameID', 'Event_Team', 'Period']).agg('mean').reset_index()

In [351]:
avgs_df['GameID'] = avgs_df['GameID'].astype(str)
combined['GameID'] = combined['GameID'].astype(str)

In [352]:
avgs_df = avgs_df.merge(combined[['Home', 'Away', 'GameID']].drop_duplicates(), how='left', on='GameID')

In [353]:
avgs_df.head()

,GameID,Event_Team,Period,x,y,Home,Away
0,2017010001,Los Angeles Kings,1,-66.888889,-6.888889,Los Angeles Kings,Vancouver Canucks
1,2017010001,Los Angeles Kings,2,56.636364,6.818182,Los Angeles Kings,Vancouver Canucks
2,2017010001,Los Angeles Kings,3,-59.500000,-7.166667,Los Angeles Kings,Vancouver Canucks
3,2017010001,Los Angeles Kings,4,71.000000,-1.666667,Los Angeles Kings,Vancouver Canucks
4,2017010001,Vancouver Canucks,1,45.428571,-1.428571,Los Angeles Kings,Vancouver Canucks


In [354]:
avgs_df['Attacking'] = avgs_df['x'].apply(lambda x: 'Right' if x > 0 else 'Left')

In [355]:
avgs_df.head()

,GameID,Event_Team,Period,x,y,Home,Away,Attacking
0,2017010001,Los Angeles Kings,1,-66.888889,-6.888889,Los Angeles Kings,Vancouver Canucks,Left
1,2017010001,Los Angeles Kings,2,56.636364,6.818182,Los Angeles Kings,Vancouver Canucks,Right
2,2017010001,Los Angeles Kings,3,-59.500000,-7.166667,Los Angeles Kings,Vancouver Canucks,Left
3,2017010001,Los Angeles Kings,4,71.000000,-1.666667,Los Angeles Kings,Vancouver Canucks,Right
4,2017010001,Vancouver Canucks,1,45.428571,-1.428571,Los Angeles Kings,Vancouver Canucks,Right


In [356]:
avgs_df['Period'] = avgs_df['Period'].astype(str)
combined['Period'] = combined['Period'].astype(str)

In [357]:
combined2 = combined.merge(avgs_df[['GameID', 'Event_Team', 'Period', 'Attacking']]
                           , how='left', on=['GameID','Event_Team', 'Period'])

In [358]:
combined2

,Assist1,Assist1_ID,Assist2,Assist2_ID,Assists2_ID,Away,Away_Score,EmptyNet,Event,EventID,Event_Description,Event_Team,Event_Type_ID,Final_Away_Score,Final_Home_Score,GameDate,GameID,GameState,GameType,Goalie,Goalie_ID,Home,Home_Score,Period,Period_Type,Player_1_ID,Player_1_Name,Player_2_ID,Player_2_Name,Scorer,Scorer_ID,Shooter,Shooter_ID,ShotType,Time_Remaining,Time_in_Period,Venue,x,y,Event2,Side,Season,Attacking
0,None,None,None,NaN,None,Vancouver Canucks,0,None,Game Scheduled,1,Game Scheduled,None,GAME_SCHEDULED,4,3,2017-09-16,2017010001,6,PR,None,None,Los Angeles Kings,0,1,REGULAR,None,None,None,None,None,None,None,None,None,20:00,00:00,STAPLES Center,NaN,NaN,Game Scheduled,NaN,2017-2018,NaN
1,None,None,None,NaN,None,Vancouver Canucks,0,None,Period Ready,54,Period Ready,None,PERIOD_READY,4,3,2017-09-16,2017010001,6,PR,None,None,Los Angeles Kings,0,1,REGULAR,None,None,None,None,None,None,None,None,None,20:00,00:00,STAPLES Center,NaN,NaN,Period Ready,NaN,2017-2018,NaN
2,None,None,None,NaN,None,Vancouver Canucks,0,None,Period Start,2,Period Start,None,PERIOD_START,4,3,2017-09-16,2017010001,6,PR,None,None,Los Angeles Kings,0,1,REGULAR,None,None,None,None,None,None,None,None,None,20:00,00:00,STAPLES Center,NaN,NaN,Period Start,NaN,2017-2018,NaN
3,None,None,None,NaN,None,Vancouver Canucks,0,None,Faceoff,3,Markus Granlund faceoff won against Adrian Kempe,Vancouver Canucks,FACEOFF,4,3,2017-09-16,2017010001,6,PR,None,None,Los Angeles Kings,0,1,REGULAR,8476440,Markus Granlund,8477960,Adrian Kempe,None,None,None,None,None,20:00,00:00,STAPLES Center,0.0,0.0,Faceoff,left,2017-2018,Right
4,None,None,None,NaN,None,Vancouver Canucks,0,None,Penalty,57,Adrian Kempe Hooking against Philip Holm,Los Angeles Kings,PENALTY,4,3,2017-09-16,2017010001,6,PR,None,None,Los Angeles Kings,0,1,REGULAR,8477960,Adrian Kempe,8480163,Philip Holm,None,None,None,None,Hooking,19:41,00:19,STAPLES Center,11.0,36.0,Penalty,right,2017-2018,Left
5,None,None,None,NaN,None,Vancouver Canucks,0,None,Faceoff,4,Bo Horvat faceoff won against Anze Kopitar,Vancouver Canucks,FACEOFF,4,3,2017-09-16,2017010001,6,PR,None,None,Los Angeles Kings,0,1,REGULAR,8477500,Bo Horvat,8471685,Anze Kopitar,None,None,None,None,None,19:41,00:19,STAPLES Center,69.0,22.0,Faceoff,left,2017-2018,Right
6,None,None,None,NaN,None,Vancouver Canucks,0,None,Blocked Shot,6,Michael Del Zotto blocked shot from Paul LaDue,Vancouver Canucks,BLOCKED_SHOT,4,3,2017-09-16,2017010001,6,PR,None,None,Los Angeles Kings,0,1,REGULAR,8474584,Michael Del Zotto,8476983,Paul LaDue,None,None,Paul LaDue,8476983,None,19:29,00:31,STAPLES Center,80.0,-6.0,Blocked Shot,left,2017-2018,Right
7,None,None,None,NaN,None,Vancouver Canucks,0,None,Blocked Shot,7,Jake Muzzin blocked shot from Troy Stecher,Los Angeles Kings,BLOCKED_SHOT,4,3,2017-09-16,2017010001,6,PR,None,None,Los Angeles Kings,0,1,REGULAR,8474162,Jake Muzzin,8479442,Troy Stecher,None,None,Troy Stecher,8479442,None,18:25,01:35,STAPLES Center,61.0,13.0,Blocked Shot,right,2017-2018,Left
8,None,None,None,NaN,None,Vancouver Canucks,0,None,Blocked Shot,8,Derek Forbort blocked shot from Bo Horvat,Los Angeles Kings,BLOCKED_SHOT,4,3,2017-09-16,2017010001,6,PR,None,None,Los Angeles Kings,0,1,REGULAR,8475762,Derek Forbort,8477500,Bo Horvat,None,None,Bo Horvat,8477500,None,17:30,02:30,STAPLES Center,64.0,11.0,Blocked Shot,right,2017-2018,Left
9,None,None,None,NaN,None,Vancouver Canucks,0,None,Hit,59,Paul LaDue hit Brock Boeser,Los Angeles Kings,HIT,4,3,2017-09-16,2017010001,6,PR,None,None,Los Angeles Kings,0,1,REGULAR,8476983,Paul LaDue,8478444,Brock Boeser,None,None,None,None,None,17:24,02:36,STAPLES Center,54.0,40.0,Hit,right,2017-2018,Left


In [359]:
def adjusted_x(row):
    if (row['Event_Team'] == row['Home']) and (row['Period'] in ['1', '3', '5']):
        if row['Attacking'] == 'Right':
            return row['x']*-1
        else:
            return row['x']
    elif (row['Event_Team'] == row['Home']) and (row['Period'] in ['2', '4']):
        if row['Attacking'] == 'Left':
            return row['x']*-1
        else:
            return row['x']
    elif (row['Event_Team'] == row['Away']) and (row['Period'] in ['1', '3', '5']):
        if row['Attacking'] == 'Left':
            return row['x']*-1
        else:
            return row['x']
    elif (row['Event_Team'] == row['Away']) and (row['Period'] in ['2', '4']):
        if row['Attacking'] == 'Right':
            return row['x']*-1
        else:
            return row['x']
    else:
        return row['x']
    
def adjusted_y(row):
    if (row['Event_Team'] == row['Home']) and (row['Period'] in ['1', '3', '5']):
        if row['Attacking'] == 'Right':
            return row['y']*-1
        else:
            return row['y']
    elif (row['Event_Team'] == row['Home']) and (row['Period'] in ['2', '4']):
        if row['Attacking'] == 'Left': 
            return row['y']*-1
        else:
            return row['y']
    elif (row['Event_Team'] == row['Away']) and (row['Period'] in ['1', '3', '5']):
        if row['Attacking'] == 'Left':
            return row['y']*-1
        else:
            return row['y']
    elif (row['Event_Team'] == row['Away']) and (row['Period'] in ['2', '4']):
        if row['Attacking'] == 'Right':
            return row['y']*-1
        else:
            return row['y']
    else:
        return row['y']

In [360]:
combined2['y_stdz'] = combined2.apply(lambda row: adjusted_y(row), axis=1)
combined2['x_stdz'] = combined2.apply(lambda row: adjusted_x(row), axis=1)

In [361]:
combined2['y_normal'] = combined2.apply(lambda row: row['y_stdz'] 
                                        if row['Period'] in ['1', '3', '5'] else row['y_stdz']*-1, axis=1)
combined2['x_normal'] = combined2.apply(lambda row: row['x_stdz'] 
                                        if row['Period'] in ['1', '3', '5'] else row['x_stdz']*-1, axis=1)

In [362]:
combined2['Event_Team_Adj'] = combined2.apply(lambda row: row['Event_Team'] if row['Event'] != 'Giveaway'
                                             else (row['Home'] if row['Event_Team'] == row['Away'] else row['Away']), axis=1)

In [363]:
combined2['y_same'] = combined2.apply(lambda row: row['y'] 
                                        if row['Side'] == 'right' else row['y']*-1, axis=1)
combined2['x_same'] = combined2.apply(lambda row: row['x'] 
                                        if row['Side'] == 'right' else row['x']*-1, axis=1)

In [365]:
#Find the 'Zones'

#Forward X >=-86, X <= -65
#Middle X > -65, X <= -51
#Back X > -51, X<=-28

#Center Y >= -3, Y <= 3
#Right Y> 3, Y<=18
#Right Board Y > 18
#Left Y<-3, Y >=-18
#Left Board Y < -18

def get_shot_zone_x(x):
    if (x >= -86) & (x <= -65):
        return "Forward"
    elif (x> -65) & (x <= -51):
        return "Middle"
    elif (x > -51) & (x <= -28):
        return "Back"
    else:
        return "Other"
        
def get_shot_zone_y(y):
    if (y >= -3) & (y <= 3):
        return "Center"
    elif (y > 3) & (y <= 18):
        return "Right"
    elif (y > 18):
        return "Right Board"
    elif (y < -3) & (y >= -18):
        return "Left"
    elif (y < -18):
        return "Left Board"
    
def get_shot_zone(row):
    if (row['zone_x'] == 'Forward') & (row['zone_y'] == 'Center'):
        return "Center Net Front"
    elif (row['zone_x'] == 'Forward') & (row['zone_y'] == 'Right'):
        return "Right Net Front"
    elif (row['zone_x'] == 'Forward') & (row['zone_y'] == 'Left'):
        return "Left Net Front"
    elif (row['zone_x'] == 'Middle') & (row['zone_y'] == 'Center'):
        return "Center Top Circle"
    elif (row['zone_x'] == 'Middle') & (row['zone_y'] == 'Left'):
        return "Left Top Circle"
    elif (row['zone_x'] == 'Middle') & (row['zone_y'] == 'Right'):
        return "Right Top Circle"
    elif (row['zone_x'] == 'Back') & (row['zone_y'] == 'Left Board'):
        return "Left Point"
    elif (row['zone_x'] == 'Back') & (row['zone_y'] == 'Right Board'):
        return "Right Point"
    elif (row['zone_x'] == 'Back') & ((row['zone_y'] == 'Center') | (row['zone_y'] == 'Left') | (row['zone_y'] == 'Right')):
        return "Center Blue Line"
    elif ((row['zone_x'] == 'Middle') | (row['zone_x'] == 'Forward')) & (row['zone_y'] == 'Left Board'):
        return "Left Board"
    elif ((row['zone_x'] == 'Middle') | (row['zone_x'] == 'Forward')) & (row['zone_y'] == 'Right Board'):
        return "Right Board"
    else:
        return "Other"
    
    
    
    

In [366]:
combined2['zone_x'] = combined2['x_same'].apply(lambda x: get_shot_zone_x(x))
combined2['zone_y'] = combined2['y_same'].apply(lambda x: get_shot_zone_y(x))

In [367]:
combined2['shot_zone'] = combined2.apply(lambda row: get_shot_zone(row), axis=1)

In [368]:
#Add an Opposition Team
combined2['Opposition Team'] = combined2.apply(lambda row:
                                              row['Away'] if row['Event_Team_Adj'] == row['Home']
                                              else row['Home'], axis=1)

In [369]:
#combined2 = pd.read_csv("combined_hockey2.csv")
combined2.to_csv("combined_hockey2.csv")

In [110]:
goaliestarted = combined2[['GameID', 'Goalie', 'Opposition Team', 'Event2']].groupby(['GameID', 'Opposition Team', 'Goalie']).count().reset_index()

In [111]:
goaliestarted = goaliestarted[goaliestarted['Goalie'] != 'None']

In [119]:
goalies = goaliestarted[['GameID', 'Opposition Team', 'Goalie']].groupby(['GameID', 'Opposition Team']).aggregate(lambda x: list(x)).reset_index()

In [124]:
goalies.columns = ['GameID', 'Home', 'Goalie']

In [127]:
combined3 = combined2.merge(goalies, on=['GameID', 'Home'], how='left', suffixes=['', '_Home'])

In [128]:
goalies.columns = ['GameID', 'Away', 'Goalie']
combined3 = combined3.merge(goalies, on=['GameID', 'Away'], how='left', suffixes=['', '_Away'])

In [129]:
combined3

,Assist 1,Assist 2,Assist1_ID,Assists2_ID,Away,Away_Score,Event,Event2,EventID,Event_Description,Event_Team,Event_Type_ID,Final_Away_Score,Final_Home_Score,GameDate,GameID,GameState,GameType,Goalie,Goalie_ID,Home,Home_Score,Period,Period_Type,Player_1_ID,Player_1_Name,Player_2_ID,Player_2_Name,Scorer,Scorer_ID,Season,Shooter,Shooter_ID,Time_Remaining,Time_in_Period,Venue,x,y,Attacking,y_stdz,x_stdz,y_normal,x_normal,Event_Team_Adj,Opposition Team,Goalie_Home,Goalie_Away
0,NaN,NaN,NaN,NaN,Vancouver Canucks,0,Game Scheduled,NaN,1,Game Scheduled,None,GAME_SCHEDULED,4,3,2017-09-16,2017010001,6,PR,NaN,NaN,Los Angeles Kings,0,1,REGULAR,None,None,None,None,NaN,NaN,2017-2018,NaN,NaN,20:00,00:00,STAPLES Center,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Los Angeles Kings,NaN,NaN
1,NaN,NaN,NaN,NaN,Vancouver Canucks,0,Period Ready,NaN,54,Period Ready,None,PERIOD_READY,4,3,2017-09-16,2017010001,6,PR,NaN,NaN,Los Angeles Kings,0,1,REGULAR,None,None,None,None,NaN,NaN,2017-2018,NaN,NaN,20:00,00:00,STAPLES Center,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Los Angeles Kings,NaN,NaN
2,NaN,NaN,NaN,NaN,Vancouver Canucks,0,Period Start,NaN,2,Period Start,None,PERIOD_START,4,3,2017-09-16,2017010001,6,PR,NaN,NaN,Los Angeles Kings,0,1,REGULAR,None,None,None,None,NaN,NaN,2017-2018,NaN,NaN,20:00,00:00,STAPLES Center,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Los Angeles Kings,NaN,NaN
3,NaN,NaN,NaN,NaN,Vancouver Canucks,0,Faceoff,NaN,3,Markus Granlund faceoff won against Adrian Kempe,Vancouver Canucks,FACEOFF,4,3,2017-09-16,2017010001,6,PR,NaN,NaN,Los Angeles Kings,0,1,REGULAR,8476440,Markus Granlund,8477960,Adrian Kempe,NaN,NaN,2017-2018,NaN,NaN,20:00,00:00,STAPLES Center,0.0,0.0,Right,0.0,0.0,0.0,0.0,Vancouver Canucks,Los Angeles Kings,NaN,NaN
4,NaN,NaN,NaN,NaN,Vancouver Canucks,0,Penalty,NaN,57,Adrian Kempe Hooking against Philip Holm,Los Angeles Kings,PENALTY,4,3,2017-09-16,2017010001,6,PR,NaN,NaN,Los Angeles Kings,0,1,REGULAR,8477960,Adrian Kempe,8480163,Philip Holm,NaN,NaN,2017-2018,NaN,NaN,19:41,00:19,STAPLES Center,11.0,36.0,Left,36.0,11.0,36.0,11.0,Los Angeles Kings,Vancouver Canucks,NaN,NaN
5,NaN,NaN,NaN,NaN,Vancouver Canucks,0,Faceoff,NaN,4,Bo Horvat faceoff won against Anze Kopitar,Vancouver Canucks,FACEOFF,4,3,2017-09-16,2017010001,6,PR,NaN,NaN,Los Angeles Kings,0,1,REGULAR,8477500,Bo Horvat,8471685,Anze Kopitar,NaN,NaN,2017-2018,NaN,NaN,19:41,00:19,STAPLES Center,69.0,22.0,Right,22.0,69.0,22.0,69.0,Vancouver Canucks,Los Angeles Kings,NaN,NaN
6,NaN,NaN,NaN,NaN,Vancouver Canucks,0,Blocked Shot,NaN,6,Michael Del Zotto blocked shot from Paul LaDue,Vancouver Canucks,BLOCKED_SHOT,4,3,2017-09-16,2017010001,6,PR,NaN,NaN,Los Angeles Kings,0,1,REGULAR,8474584,Michael Del Zotto,8476983,Paul LaDue,NaN,NaN,2017-2018,NaN,NaN,19:29,00:31,STAPLES Center,80.0,-6.0,Right,-6.0,80.0,-6.0,80.0,Vancouver Canucks,Los Angeles Kings,NaN,NaN
7,NaN,NaN,NaN,NaN,Vancouver Canucks,0,Blocked Shot,NaN,7,Jake Muzzin blocked shot from Troy Stecher,Los Angeles Kings,BLOCKED_SHOT,4,3,2017-09-16,2017010001,6,PR,NaN,NaN,Los Angeles Kings,0,1,REGULAR,8474162,Jake Muzzin,8479442,Troy Stecher,NaN,NaN,2017-2018,NaN,NaN,18:25,01:35,STAPLES Center,61.0,13.0,Left,13.0,61.0,13.0,61.0,Los Angeles Kings,Vancouver Canucks,NaN,NaN
8,NaN,NaN,NaN,NaN,Vancouver Canucks,0,Blocked Shot,NaN,8,Derek Forbort blocked shot from Bo Horvat,Los Angeles Kings,BLOCKED_SHOT,4,3,2017-09-16,2017010001,6,PR,NaN,NaN,Los Angeles Kings,0,1,REGULAR,8475762,Derek Forbort,8477500,Bo Horvat,NaN,NaN,2017-2018,NaN,NaN,17:30,02:30,STAPLES Center,64.0,11.0,Left,11.0,64.0,11.0,64.0,Los Angeles Kings,Vancouver Canucks,NaN,NaN
9,NaN,NaN,NaN,NaN,Vancouver Canucks,0,Hit,NaN,59,Paul LaDue hit Brock Boeser,Los Angeles Kings,HIT,4,3,2017-09-16,2017010001,6,PR,NaN,NaN,Los Angeles Kings,0,1,REGULAR,8476983,Paul LaDue,8478444,Brock Boeser,NaN,NaN,2017-2018,NaN,NaN,17:24,02:36,STAPLES Center,54.0,40.0,Left,40.0,54.0,40.0,54.0,Los Angeles Kings,Vancouver Canucks,NaN,NaN


In [131]:
combined3.to_csv("combined2.csv", index=False)

In [ ]:
avgs_df[avgs_df['GameID'] == '2017021171']

In [132]:
goalies.head()

,GameID,Away,Goalie
0,2018010002,Boston Bruins,[Zane McIntyre]
1,2018010002,Washington Capitals,"[Pheonix Copley, Vitek Vanecek]"
2,2018010003,New York Islanders,"[Christopher Gibson, Robin Lehner]"
3,2018010003,Philadelphia Flyers,"[Alex Lyon, Anthony Stolarz]"
4,2018010004,Arizona Coyotes,"[Adin Hill, Darcy Kuemper]"
